dkernal1<<<1D,1D>>> //Addition of 2D matrices uisng 1D Grid ,1D Block thread organization (Single Thread Per Row)

In [ ]:
//KERNEL_1 for Assignment1
__global__ void dkernal1(int *ga,int *gb,int *gc,int n,int m){
    int id;
    id=blockIdx.x*blockDim.x+threadIdx.x;
    if(id<n){
          for(int j=0;j<m;j++){
            gc[id*m+j]=ga[id*m+j]+gb[id*m+j];
        }  
    }
    
}

dkernal2<<<1D,2D>>> //Addition of 2D matrices uisng 1D Grid ,2D Block thread organization (Single Thread Per Column)

In [ ]:
__global__ void dkernal2(int *ga,int *gb,int *gc,int n,int m){
    int id;
    id =blockDim.x*blockDim.y*blockIdx.x+ blockDim.x* threadIdx.y+threadIdx.x;
    if(id<m){
          for(int j=0;j<n;j++){
            gc[id*n+j]=ga[id*n+j]+gb[id*n+j];
        }  
    }
    
}

dkernal3<<<2D,2D>>> //Addition of 2D martces uisng 2D Grid ,2D Block thread organization (Single Thread per Element)

In [ ]:
__global__ void dkernal3(int *ga,int *gb,int *gc,int n,int m){
    int i,j,id;
    i = threadIdx.x + blockDim.x * threadIdx.y;
    j = blockIdx.x + gridDim.x * blockIdx.y;
    id=j*blockDim.x*blockDim.y+i;
    if(id<(m*n))
    gc[id]=ga[id]+gb[id];
    
}

In [ ]:
!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-qr03lth7
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-qr03lth7
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=c3a0c5950048b8b36a1c29a0572ea00cc7551d93e6ae809c0a93930501fda626
  Stored in directory: /tmp/pip-ephem-wheel-cache-ox9kw4qw/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cuda -n  Ass1.cu
#include<iostream>
#include<cuda.h>
#include <fstream>
#include <string>
#define blocks 1024
using namespace std;
__global__ void dkernal(int *ga,int *gb,int *gc,int n,int m){
    int id;
    id=blockIdx.x*blockDim.x+threadIdx.x;
    if(id<n){
          for(int j=0;j<m;j++){
            gc[id*m+j]=ga[id*m+j]+gb[id*m+j];
        }  
    }
    
}
int main(){
    
    int n,m,it;
    int *a,*b,*c,*ga,*gb,*gc,*d;
    string line; 
    cin>>n>>m;
    cudaMalloc(&ga,(n*m)*sizeof(int));
    cudaMalloc(&gb,(n*m)*sizeof(int));
    cudaMalloc(&gc,(n*m)*sizeof(int));
    a=(int*)malloc(n*m*sizeof(int));
    b=(int*)malloc(n*m*sizeof(int));
    c=(int*)malloc(n*m*sizeof(int));
    d=(int*)malloc(n*m*sizeof(int));
    
                ifstream myfile ("/content/src/input1.txt");
                if (myfile.is_open()){                     //if the file is open
                                                            //while the end of file is NOT reached
                      //getline (myfile2,line);                //get one line from the file
                          for (int i=0; i<n;i++){         //criteria for input loop (as array 0-5)
                              for (int j=0; j<m;j++){
                                  myfile>>it;
                                  a[i*m+j]=it;
                              }                         
                          }
                          for (int i=0; i<n;i++){         //criteria for input loop (as array 0-5)
                              for (int j=0; j<m;j++){
                                  myfile>>it;
                                  b[i*m+j]=it;
                              }                         
                          }
                    myfile.close();                         //closing the file
                  }                  
                  ifstream myfile2 ("/content/src/output1.txt");
                  if (myfile2.is_open()){                     //if the file is open
                                                              //while the end of file is NOT reached
                        //getline (myfile2,line);                //get one line from the file
                            for (int i=0; i<n;i++){         //criteria for input loop (as array 0-5)
                                for (int j=0; j<m;j++){
                                    myfile2>>it;
                                    d[i*m+j]=it;
                                }                         
                            }
                          
                      myfile2.close();                         //closing the file
                    }

   
    cudaMemcpy(ga,a,n*m*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(gb,b,n*m*sizeof(int),cudaMemcpyHostToDevice);
    //dim3 nblo(32,32,1);
    //dim3 grid(64,32,1);
    dim3 bloc(25,25,1);
    //unsigned int gr =ceil((float)n/blocks);
    //dkernal<<<1,block>>>(ga,gb,gc,n,m);
    
    dkernal<<<40,2>>>(ga,gb,gc,n,m);
    cudaDeviceSynchronize();

    cudaMemcpy(c,gc,n*m*sizeof(int),cudaMemcpyDeviceToHost);
    int cx=0;
    for(int i=0;i<n;i++){
        for(int j=0;j<m;j++){
            cout<<c[i*m+j]-d[i*m+j]<<" ";
            if(c[i*m+j]-d[i*m+j]!=0){
                cx+=1;
            }
        }
        cout<<"\n";
    }
    cout<<" non zero elements :"<<cx;
    
    return 0;
}

'File written in /content/src/Ass1.cu'

In [ ]:
!nvcc /content/src/Ass1.cu

/content/src/Ass1.cu(68): warning: variable "bloc" was declared but never referenced



In [ ]:
!./a.out

41 2083
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
%%cuda -n  Ass1.cu
#include<iostream>
#include<cuda.h>
#include <fstream>
#include <string>
#define blocks 1024
using namespace std;
__global__ void dkernal(int *ga,int *gb,int *gc,int n,int m){
    int i,j,id;
    i = threadIdx.x + blockDim.x * threadIdx.y;
    j = blockIdx.x + gridDim.x * blockIdx.y;
    id=j*blockDim.x*blockDim.y+i;
    if(id<(m*n))
    gc[id]=ga[id]+gb[id];
    
}
int main(){
    
    int n,m,it;
    int *a,*b,*c,*ga,*gb,*gc,*d;
    string line; 
    cin>>n>>m;
    cudaMalloc(&ga,(n*m)*sizeof(int));
    cudaMalloc(&gb,(n*m)*sizeof(int));
    cudaMalloc(&gc,(n*m)*sizeof(int));
    a=(int*)malloc(n*m*sizeof(int));
    b=(int*)malloc(n*m*sizeof(int));
    c=(int*)malloc(n*m*sizeof(int));
    d=(int*)malloc(n*m*sizeof(int));
    
                ifstream myfile ("/content/src/input1_M_1.txt");
                if (myfile.is_open()){                     //if the file is open
                                                            //while the end of file is NOT reached
                      //getline (myfile2,line);                //get one line from the file
                          for (int i=0; i<n;i++){         //criteria for input loop (as array 0-5)
                              for (int j=0; j<m;j++){
                                  myfile>>it;
                                  a[i*m+j]=it;
                              }                         
                          }
                    myfile.close();                         //closing the file
                  }


                  ifstream myfile1 ("/content/src/input1_M_2txt.txt");
                  if (myfile1.is_open()){                     //if the file is open
                                                              //while the end of file is NOT reached
                        //getline (myfile1,line);                //get one line from the file
                            for (int i=0; i<n;i++){         //criteria for input loop (as array 0-5)
                                for (int j=0; j<m;j++){
                                    myfile1>>it;
                                    b[i*m+j]=it;
                                }                         
                            }
                          
                      myfile1.close();                         //closing the file
                    }

                  
                  ifstream myfile2 ("/content/src/output1.txt");
                  if (myfile2.is_open()){                     //if the file is open
                                                              //while the end of file is NOT reached
                        //getline (myfile2,line);                //get one line from the file
                            for (int i=0; i<n;i++){         //criteria for input loop (as array 0-5)
                                for (int j=0; j<m;j++){
                                    myfile2>>it;
                                    d[i*m+j]=it;
                                }                         
                            }
                          
                      myfile2.close();                         //closing the file
                    }

   
    cudaMemcpy(ga,a,n*m*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(gb,b,n*m*sizeof(int),cudaMemcpyHostToDevice);
    dim3 nblo(32,32,1);
    dim3 grid(25,32,1);
    //unsigned int gr =ceil((float)n/blocks);
    dkernal<<<grid,nblo>>>(ga,gb,gc,n,m);
    cudaDeviceSynchronize();

    cudaMemcpy(c,gc,n*m*sizeof(int),cudaMemcpyDeviceToHost);
    int cx=0;
    for(int i=0;i<n;i++){
        for(int j=0;j<m;j++){
            cout<<c[i*m+j]-d[i*m+j]<<" ";
            if(c[i*m+j]-d[i*m+j]!=0){
                cx+=1;
            }
        }
        cout<<"\n";
    }
    cout<<" non zero elements :"<<cx;
    
    return 0;
}

'File written in /content/src/Ass1.cu'

In [ ]:
%%cuda -n  TEST.cu
#include<iostream>
#include<cuda.h>
using namespace std;
__global__ void dkernal(int *ga,int *gb,int *gc,int n,int m){
    int i,j;
    i=blockIdx.x*blockDim.x+threadIdx.x;
    j=blockIdx.y*blockDim.y+threadIdx.y;
    gc[i*m+j]=ga[i*m+j]+gb[i*m+j];
}
int main(){
    
    int n,m;
    int *a,*b,*c,*ga,*gb,*gc;
    cin>>n>>m;
    cudaMalloc(&ga,(n*m)*sizeof(int));
    cudaMalloc(&gb,(n*m)*sizeof(int));
    cudaMalloc(&gc,(n*m)*sizeof(int));
    a=(int*)malloc(n*m*sizeof(int));
    b=(int*)malloc(n*m*sizeof(int));
    c=(int*)malloc(n*m*sizeof(int));
    for(int i=0;i<n;i++){
        for(int j=0;j<m;j++){
            int t;
            scanf("%d",&t);
            a[i*m+j]=t;
        }
    }    
    for(int i=0;i<n;i++){
        for(int j=0;j<m;j++){
            int t;
            scanf("%d",&t);
            b[i*m+j]=t;
        }
    }
    cudaMemcpy(ga,a,n*m*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(gb,b,n*m*sizeof(int),cudaMemcpyHostToDevice);
    dim3 block(n,m,1);
    dkernal<<<1,block>>>(ga,gb,gc,n,m);
    cudaMemcpy(c,gc,n*m*sizeof(int),cudaMemcpyDeviceToHost);
    for(int i=0;i<n;i++){
        for(int j=0;j<m;j++){
            cout<<c[i*m+j]<<" ";
        }
        cout<<"\n";
    }
    //dkernal<<<1,block>>>(n);
    cudaDeviceSynchronize();
    return 0;
}

'File written in /content/src/Inrun.cu'

In [ ]:
!nvcc /content/src/Inrun.cu

In [ ]:
!./a.out

2 3
1 2 3 4 5 6
2 2 2 2 2 2
3 4 5 
6 7 8 


In [ ]:
!git --version

git version 2.17.1


In [ ]:
!cd /content/src

In [ ]:
!pwd

/content


In [ ]:
from getpass import getpass
password=getpass('Password:')

Password:··········


In [ ]:
!git config --global user.email "sashedher.14@gmail.com"
!git config --global user.name "sashedher"

In [ ]:
!git remote add origin https://<sashedher>:$password@github.com/<sashedher>/https://github.com/sashedher/GPU_Programming.git

/bin/bash: sashedher: No such file or directory


In [ ]:
!git init

Reinitialized existing Git repository in /content/.git/


In [ ]:
!git status

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.config/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
!!git add /content/src/.
!git add a.out

In [ ]:
!git commit -m "Final code for Assign_1"

[master (root-commit) 32cb067] Final code for Assign_1
 10 files changed, 2379 insertions(+)
 create mode 100755 a.out
 create mode 100644 src/Ass1.cu
 create mode 100644 src/input1.txt
 create mode 100644 src/input1_M_1.txt
 create mode 100644 src/input1_M_2txt.txt
 create mode 100644 src/input2.txt
 create mode 100644 src/input3.txt
 create mode 100644 src/output1.txt
 create mode 100644 src/output2.txt
 create mode 100644 src/output3.txt


In [ ]:
!git push -u origin https://github.com/sashedher/GPU_Programming.git

fatal: remote part of refspec is not a valid name in https://github.com/sashedher/GPU_Programming.git


In [ ]:
!git pull origin master https://github.com/sashedher/GPU_Programming.git

fatal: Invalid refspec 'https://github.com/sashedher/GPU_Programming.git'


In [ ]:
!git push -u origin master

fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!git remote rm origin

fatal: No such remote: origin
